In [35]:
import requests
files = {'file': open('gun/75.png','rb')}
r = requests.post('http://127.0.0.1:8000/uploaded_images', files=files)
print(r.text)

{"result":{"status":"success","response":"{\"man\": 1, \"gun\": 0}"}}


In [32]:
r = requests.get('http://127.0.0.1:8000/')
print(r.text)

{"Hello":"World"}


В этой программе мы создаем класс `FolderChecker`, который имеет два метода: `__init__` для инициализации и `check_new_files` для проверки на наличие новых файлов. Метод `__init__` принимает путь к папке в качестве аргумента и создает объект класса.

Метод `check_new_files` использует модуль `os` для получения списка файлов в папке и обновляет последний раз, когда был изменен каждый файл. Если был найден новый файл, выполняется POST-запрос с помощью модуля `requests` на локальный веб-сервер с файлом, который был добавлен в папку.

В конце программы мы создаем объект класса `FolderChecker` и запускаем цикл бесконечного ожидания новых файлов. Когда новый файл будет обнаружен, мы выполним POST-запрос.

In [ ]:
import os
import requests
from datetime import datetime

class FolderChecker:
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self._files = {}
        self._last_modified_time = datetime.now()

    def check_new_files(self):
        if "last_modified" in self._files.keys():
            last_modified = self._files["last_modified"]
        else:
            last_modified = ""
        # Проверяем наличие новых файлов в папке
        for filename in os.listdir(self.folder_path):
            if filename not in self._files:
                # Если файл не был ранее добавлен в список, добавляем его
                self._files[filename] = datetime.fromtimestamp(os.path.getctime(os.path.join(self.folder_path, filename)))
                # Обновляем последнее время модификации файла
                self._last_modified_time = max(self._last_modified_time, self._files[filename])

        # Если были обнаружены новые файлы, выполняем POST-запрос на локальный сервер
        if self._last_modified_time > self._files["last_modified"]:
            url = "http://127.0.0.1:8000/uploaded_images"
            data = {"file": open(self.folder_path + "/newfile.txt", "rb")}
            requests.post(url, data=data)

    # Вызываем метод для проверки новых файлов
    def run(self):
        while True:
            self.check_new_files()
            sleep(2)

if __name__ == "__main__":
    folder_checker = FolderChecker("gun")
    folder_checker.run()

сделать bat файл

In [ ]:
!pip freeze > requarements.txt 

Эта функция принимает путь к папке, где нужно проверять наличие новых файлов. Она создает словарь file_info, где ключи - это имена файлов, а значения - информация о каждом файле. Затем она проверяет, есть ли в словаре ключ newfile.txt, и если есть, то сравнивает время последнего изменения этого файла с временем последнего изменения всех остальных файлов. Если newfile.txt был добавлен после других файлов, то функция добавляет его имя в список new_files. Наконец, если есть новые файлы, функция отправляет POST-запрос на указанный URL с данными, содержащими список новых файлов.

In [ ]:
import os
from datetime import datetime
import requests

def check_folder(folder_path):  # Функция проверки папки
    # Получаем список файлов в папке
    files = os.listdir(folder_path)
    # Создаем словарь для хранения информации о файлах
    file_info = {}
    for file in files:
        file_info[file] = {
            "name": file,
            "last_modified": datetime.now(),
        }
    # Проверяем, есть ли новые файлы в папке
    new_files = []
    for filename, info in file_info.items():
        if info["last_modified"] > file_info["newfile.txt"]["last_modified"]:  # Если файл был добавлен после нового файла
            new_files.append(filename)
    if new_files:  # Если есть новые файлы
        url = "http://127.0.0.1:8000/uploaded_images"  # URL для передачи данных
        data = {"files": new_files}  # Данные для передачи
        requests.post(url, json=data)  # Отправляем POST-запрос

In [ ]:
import os, requests
from PIL import Image
from io import BytesIO

# Путь к папке с картинками
folder_path = "gun"

# Функция для проверки папки на наличие новых изображений
def check_images():
    # Список изображений в папке
    images = [os.path.join(folder_path, image) for image in os.listdir(folder_path)]

    # Создание пустого словаря для хранения информации об изображениях
    image_info = {}

    for image in images:
        with Image.open(image) as img:
            # Получение размера изображения
            width, height = img.size

            # Добавление информации об изображении в словарь
            info = {
                "width": width,
                "height": height,
                "size": img.size,
                "file_name": os.path.basename(image),
            }
            image_info[os.path.splitext(image)[0]] = info

    # Проверка на наличие новых изображений в словаре
    new_images = []
    for key, value in image_info.items():
        if value["file_name"] != key:
            new_images.append(key)

    if len(new_images) > 0:
        # Создание объекта запроса
        files = {'file': open(f'{new_images}.png','rb')}
        r = requests.post("http://127.0.0.1:8000/uploaded_images", files=files)
        print("Отправлен POST-запрос")


# Запуск функции проверки
check_images()